# Forecasting with MCForecast

In this activity you will use the provided `MCForecastTools.py` module to forecast a portfolio's returns for one years worth of stock data from Home Depot and Disney.
  - You will retrieve data using the Quandl API.
  - You will then concatenate the dataframes for use with `MCSimulation`.
  - Then you will use `calc_cumulative_return` to get the cumulative return for the year.
  - After that you will plot the simulations, examine the distribution and review the summary statistics for the simulation.

>**Hint**: The most authoritative documentation you can find is the source code itself. You will need to read through the `MCForecastTools.py` file to get an idea of how the module is working.

## 1. Import Modules

In [1]:
# Import modules.
import pandas as pd
import numpy.random as rnd
from dotenv import load_dotenv
import requests
import os
import json
from MCForecastTools import MCSimulation


## 2. Load Environment Variables

In [2]:
# Load .env enviroment variables into the notebook
load_dotenv('../../../../.env')

# Get the API key from the environment variable and store it as a python variable
quandl_api_key = os.getenv('QUANDL_API_KEY_ENV')

## 3. Get Data from Quandl
  - Use the provided tickers and urls

In [3]:

# Ticker symbols
home_depot = "HD"
disney = "DIS"

In [4]:
# Urls for Quandl
url_hd = f"https://www.quandl.com/api/v3/datasets/WIKI/{home_depot}.json?&api_key={quandl_api_key}"

url_dis = f"https://www.quandl.com/api/v3/datasets/WIKI/{disney}.json?&api_key={quandl_api_key}"

In [5]:
# Home Depot response
hd_response = requests.get(url_hd).json()

In [29]:
# Disney response
dis_response = requests.get(url_dis).json()
print(json.dumps(dis_response, indent=4, sort_keys=True))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## 4. Create a `pandas` `DataFrame`
- Using the response json objects, get a list of the column names and create dataframes for each ticker.
- Convert each dataframe's `Date` field to a `datetime` object and set `Date` as the index.
  - Use `to_datetime` to convert a string into a datetime object.
  - Use the `infer_datetime_format` parameter.
- Filter each dataframe to only keep the `"Open", "High", "Low", "Close", "Volume"` columns.
- The columns will then need to be renamed using all lowercase letters to work with `MCForecastTools`.
- Create a dictionary to hold the Home Depot and Disney dataframes similar to the following:
  ```
  { "ticker_name": ticker_df , .... }
  ```
  `MCForecastTools` requires that the dataframe be multi-indexed.
- Use the dictionary and `concat` from pandas to concentate the dataframes and return a multi-indexed dataframe.
- Use the dictionary's `.value()` method for the data parameter into `concat`.
- Set `axis = 1`.
- Use the dictionary's `.keys()` method for the `keys` parameter in `concat`.

In [7]:
# Get list of column names
# YOUR CODE HERE!

In [8]:
# Create a dataframe from Home Depot the response.
# YOUR CODE HERE!

In [9]:
# Convert `Date` to datetime
# YOUR CODE HERE!


In [10]:
# Set index
# YOUR CODE HERE!

In [11]:
# Print Home Depot dataframe
# YOUR CODE HERE!

In [12]:
# Create a dataframe from the Disney response.
# YOUR CODE HERE!

In [13]:
# Convert `Date` to datetime
# YOUR CODE HERE!


In [14]:
# Set index
# YOUR CODE HERE!

In [15]:
# Print Disney dataframe
# YOUR CODE HERE!

In [16]:
# Take only the "Open", "High", "Low", "Close", "Volume" columns for each dataframe.
# YOUR CODE HERE!

In [17]:
# MCSimulation is case sensitive to column names.
# Rename the columns to "open", "high", "low", "close", "volume"
# YOUR CODE HERE!

In [18]:
# Create a dictionary of the two dataframes
# YOUR CODE HERE!

In [19]:
# Use concat to create a merged dataframe from the dictionary
# YOUR CODE HERE!


In [20]:
# YOUR CODE HERE!


## 5. Forecast with `MCSimulation`
- Create an instance of `MCSimulation` with the following parameter values:
  - `weights == [0.25,0.75]`
  - `num_simulations == 500`
  - `num_trading_days == 252`
- Call the `calc_cumulative_return` method to run the number of simulations.
- Call the `plot_simulation` method to plot the simulated returns.
- Call `plot_distribution` to plot a histogram of the underlying distribution.
- Call `summarize_cumulative_return` to display summary statistics for the simulated distribution.

In [21]:
# Create a simulation object
# YOUR CODE HERE!


In [22]:
# Review the simulation input data
# YOUR CODE HERE!

In [23]:
# Run the Monte Carlo simulation to forecast the cumulative returns
# YOUR CODE HERE!

In [24]:
# Visualize the simulation by creating an
# overlay line plot
# YOUR CODE HERE!

In [25]:
# Visualize the probability distribution of the simulation
# by plotting a histogram
# YOUR CODE HERE!

In [26]:
# Generate summary statistics from the simulation results
# Save the results as a variable
# YOUR CODE HERE!

# Review the summary statistics
# YOUR CODE HERE!
